# MVP Engenharia de Dados
### Viviane Mendes Matos - Matrícula 

# 1. Objetivo
O objetivo deste MVP é responder algumas questões importantes sobre o conteúdo da plataforma Disney+ conhecida mundialmente e amplamente consumida por pessoas de todas as idades.

O Disney+ é uma das plataformas de streaming de vídeo e mídia mais populares. Com cerca de 1450 filmes e séries disponíveis em sua plataforma (dados de meados de 2021), a plataforma contava com mais de 116 milhões de assinantes em todo o mundo. Este conjunto de dados tabulares lista todos os filmes e séries disponíveis no Disney+, incluindo detalhes como elenco, diretores, classificação indicativa, ano de lançamento, duração, etc.

Estas questões principais são:
1) Qual é o tempo médio entre o lançamento de um filme no cinema e entrada deste mesmo conteúdo no catálogo de streaming do Disney+?
2) Como foi a evolução da diversidade de gêneros no catálogo ao longo do tempo?
3) Quais gêneros são mais comuns no conteúdo do Disney+?

# 2. Coleta de Dados
A fim de responder às hipóteses levantadas, foi selecionado o dataset 'Disney+ Movies and TV Shows' disponível no Kaggle (link: https://www.kaggle.com/datasets/shivamb/disney-movies-and-tv-shows ) e hospedado no GitHub no link (https://github.com/vivianemattos/mvp-vmm-engenharia-de-dados/blob/main/data/disney_plus_titles_w43.csv).

Foi realizado download do dataset e carregado via python no notebook.



#3. Arquitetura
Para este trabalho foi selecionada a arquitetura Medallion implementada no Databricks Free Edition. 

A arquitetura Medallion no Databricks é um padrão de organização de dados (muito usado com Delta Lake) que estrutura o pipeline em camadas para evoluir os dados de “brutos” para “prontos para consumo”, com qualidade, rastreabilidade e governança ao longo do caminho. Em vez de “um ETL gigante”, você tem etapas claras e auditáveis.

- **Camada Bronze (ingest)**: Entrada de dados, onde são capturados e preservados os dados originais com pouquíssima ou nenhuma transformação
- **Camada Silver (clean)**: Dados limpos, padronizados e consistentes
- **Camada Gold (analytics)**: Camada final para consumo, dados modelados para o negócio e prontos para serem consumidos em ferramentas de BI ou relatórios

# 4. Modelagem

Foi feita a modelagem em esquema estrela para refletir a necessidade de resposta às hipóteses levantadas como objetivo deste MVP de forma a otimizar as informações para geração de relatórios e dashboards.

![](https://github.com/vivianemattos/mvp-vmm-engenharia-de-dados/blob/main/notebooks/MER%20StarSchema%20Disney%2B.png)

##4.1 Catálogo de Dados



#5. Carga de Dados
Neste espaço temos a ingestão de dados direto da fonte com leitura e persistência dos dados na tabela Delta.


##5.1 Camada de Preparação / Setup

In [0]:
#CAMADA DE PREPARAÇÃO
## Import de Bibliotecas
## Criação e Estruturas de Schemas

#--IMPORTANDO BIBLIOTECAS--

%pip install requests

import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.types import (StructType, StructField, StringType, DateType, IntegerType)
#import dlt
from pyspark.sql.functions import col, trim, to_date, regexp_extract, when, split, explode
import requests
from pyspark.sql import functions as F


#--CAMADA BRONZE--
#criacao do schema bronze
spark.sql(""" CREATE SCHEMA IF NOT EXISTS bronze """)

#definicao do esquema bronze personalizado
bronze = StructType([
    StructField("show_id", StringType(), True), 
    StructField("type", StringType(), True),         
    StructField("director", StringType(), True),
    StructField("cast", StringType(), True), 
    StructField("country", StringType(), True), 
    StructField("date_added", DateType(), True),
    StructField("release_year", IntegerType(), True),
    StructField("rating", StringType(), True),
    StructField("duration", StringType(), True),
    StructField("listed_in", StringType(), True),
    StructField("description", StringType(), True)
])

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


##5.2 Camada Bronze (ingest)
Realizar a ingestão do dataset Disney+ Movies and TV Shows a partir de um arquivo CSV versionado no GitHub.

Nesta camada, os dados são carregados **exatamente como recebidos**, sem transformações semânticas, garantindo rastreabilidade e reprodutibilidade.

O arquivo é lido diretamente do repositório GitHub, garantindo que qualquer execução futura utilize exatamente a mesma versão do dataset.

In [0]:
# ## Leitura do Arquivo CSV a partir do GitHub e Criação de Tabela Bronze para Persistência dos Dados

#--INGESTÃO DOS DADOS BRUTOS NA CAMADA BRONZE

# Leitura do arquivo csv fonte para um dataframe pandas e spark
import pandas as pd
from pyspark.sql.functions import *

url = "https://raw.githubusercontent.com/vivianemattos/mvp-vmm-engenharia-de-dados/main/data/disney_plus_titles_w43.csv"
pd_df_bronze_raw = pd.read_csv(url)

#conversão para spark
spark_df_bronze_raw = spark.createDataFrame(pd_df_bronze_raw)

#criação da tabela bronze para persistência dos dados
spark_df_bronze_raw.write.format("delta").mode("overwrite").saveAsTable("bronze.disney_titles_raw")

#criação de view temporária para ser referenciada em outros notebooks
spark_df_bronze_raw.createOrReplaceTempView("temp_df_bronze")

#exibição dos 5 primeiros registros da tabela bronze
#display(spark_df_bronze_raw)
print("Exibição dos 5 primeiros registros da tabela bronze")
display(spark_df_bronze_raw.limit(5))

#exibição dos 5 últimos registros da tabela bronze
print("Exibição dos 5 últimos registros da tabela bronze")
display(spark_df_bronze_raw.orderBy(desc("show_id")).limit(5))

Exibição dos 5 primeiros registros da tabela bronze


show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacNeille, Bill Farmer, Russi Taylor, Corey Burton",null,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",null,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. Nic.
s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis Leary, Queen Latifah",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexander Jean, Fall Out Boy, Jimmie Allen",null,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
s5,TV Show,The Beatles: Get Back,null,"John Lennon, Paul McCartney, George Harrison, Ringo Starr",null,"November 25, 2021",2021,null,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson capturing a moment in music history with The Beatles.


Exibição dos 5 últimos registros da tabela bronze


show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
s999,Movie,Lady and the Tramp,Charlie Bean,"Tessa Thompson, Justin Theroux, Sam Elliott, Ashley Jensen, Janelle Monáe, Kiersey Clemons",United States,"November 12, 2019",2019,PG,111 min,"Action-Adventure, Comedy, Family",A house dog and a stray embark on an adventure and come to understand the value of home.
s998,TV Show,Lab Rats,null,"Billy Unger, Spencer Boldman, Kelli Berglund, Tyrel Williams, Hal Sparks",United States,"November 12, 2019",2011,TV-Y7,4 Seasons,"Action-Adventure, Comedy, Coming of Age",Leo discovers a secret underground lab with three experiments.
s997,Movie,La Luna,Enrico Casarosa,"Tony Fucile, Krista Sheffler, Phil Sheridan",United States,"November 12, 2019",2012,G,7 min,"Animation, Coming of Age, Family",A peculiar story about a young boy coming of age.
s996,Movie,Kronk's New Groove,"Elliot Bour, Saul Blinkoff","Patrick Warburton, Tracey Ullman, Eartha Kitt, David Spade, John Goodman, Wendie Malick",United States,"November 12, 2019",2005,G,78 min,"Animation, Comedy, Coming of Age",Kronk cooks up a get-rich-quick scheme to impress his Dad.
s995,Movie,Knick Knack Theatrical Short,John Lasseter,null,null,"November 12, 2019",2003,G,4 min,"Animation, Family, Kids",A snowman longs to break out of his snow globe.


### Validações Iniciais

Nesta etapa são realizadas validações básicas para garantir:
- leitura correta do arquivo
- quantidade esperada de registros
- estrutura compatível com o dataset original


In [0]:
%sql
SELECT 
    COUNT(*) AS total_registros
FROM bronze.disney_titles_raw;

total_registros
1450


#5.3 Camada Silver (cleaning: Tratamento e Padronização)

Transformar os dados da camada Bronze em uma base canônica (Silver), com:

- padronização de tipos (ex.: datas)
- normalização de atributos (ex.: duração em minutos / temporadas)
- normalização relacional de campos multivalorados (N:N), garantindo melhor qualidade analítica

### Fonte
- Entrada: `bronze.disney_titles_raw`
- Saídas:
  - `silver.disney_titles_clean` (tabela canônica)
  - tabelas normalizadas:
    - `silver.title_genres`
    - `silver.title_countries`
    - `silver.title_directors`
    - `silver.title_cast`

### Principais Transformações
- Conversão de `date_added` para tipo data
- Normalização de duração (minutos / temporadas)
- Padronização de campos categóricos

In [0]:
from pyspark.sql import functions as F

df = spark.table("bronze.disney_titles_raw")

print("1. Iniciando transformações de campos com padronizações leves")
# Padronizações leves e canônicas
df_silver = (
    df
    # trims básicos
    .withColumn("show_id", F.trim(F.col("show_id")))
    .withColumn("type", F.trim(F.col("type")))
    .withColumn("title", F.trim(F.col("title")))
    .withColumn("rating", F.trim(F.col("rating")))
    .withColumn("listed_in", F.trim(F.col("listed_in")))
    .withColumn("country", F.trim(F.col("country")))
    .withColumn("director", F.trim(F.col("director")))
    .withColumn("cast", F.trim(F.col("cast")))
    .withColumn("description", F.trim(F.col("description")))
    
    # parse de data
    .withColumn("date_added_dt", F.to_date("date_added", "MMMM d, yyyy"))
    
    # normalização de duração (num/unidade)
    .withColumn("duration_num", F.regexp_extract("duration", r"(\d+)", 1).cast("int"))
    .withColumn(
        "duration_unit",
        F.when(F.col("duration").contains("min"), F.lit("min"))
         .when(F.col("duration").contains("Season"), F.lit("season"))
         .otherwise(F.lit(None))
    )
    .withColumn("duration_minutes", F.when(F.col("duration_unit") == "min", F.col("duration_num")))
    .withColumn("seasons", F.when(F.col("duration_unit") == "season", F.col("duration_num")))
    
    # tipagem do ano de lançamento
    .withColumn("release_year_int", F.col("release_year").cast("int"))
    
    # normalização de strings vazias para null (boa prática)
    .withColumn("director", F.nullif(F.col("director"), F.lit("")))
    .withColumn("cast", F.nullif(F.col("cast"), F.lit("")))
    .withColumn("country", F.nullif(F.col("country"), F.lit("")))
    .withColumn("rating", F.nullif(F.col("rating"), F.lit("")))
)

print("Transformações de campos com padronizações leves concluídas (Remoção de vazios, normalização de duração, conversão de data, tipo do ano de lançamento e normalização de strings vazias para null).")

# Remover duplicidades por show_id (se existirem) mantendo a primeira ocorrência
print("2. Removendo duplicadas no campo show_id")
df_silver = df_silver.dropDuplicates(["show_id"])
print("Duplicadas no campo show_id removidas.")

# Persistência canônica Silver
print("3. Persistindo dados canônicos camada Silver em tabela Delta")
(df_silver
 .write
 .format("delta")
 .mode("overwrite")
 .saveAsTable("silver.disney_titles_clean"))
print("Tabela Delta salva")

print("4. Exibindo 5 primeiras linhas da tabela Delta - Camada Silver")
display(df_silver.limit(5))

1. Iniciando transformações de campos com padronizações leves
Transformações de campos com padronizações leves concluídas (Remoção de vazios, normalização de duração, conversão de data, tipo do ano de lançamento e normalização de strings vazias para null).
2. Removendo duplicadas no campo show_id
Duplicadas no campo show_id removidas.
3. Persistindo dados canônicos camada Silver em tabela Delta
Tabela Delta salva
4. Exibindo 5 primeiras linhas da tabela Delta - Camada Silver


show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,date_added_dt,duration_num,duration_unit,duration_minutes,seasons,release_year_int
s537,TV Show,Disney Prop Culture,null,Daniel Lanigan,United States,"May 1, 2020",2020,TV-PG,1 Season,Docuseries,"Collector Dan Lanigan reunites iconic Disney movie props with filmmakers, actors, and crew.",2020-05-01,1,season,null,1,2020
s722,Movie,Alice in Wonderland,"Clyde Geronimi, Hamilton Luske, Wilfred Jackson","Ed Wynn, Richard Haydn, Sterling Holloway, Jerry Colonna, Verna Felton, Pat O'Malley",United States,"November 12, 2019",1951,G,79 min,"Action-Adventure, Animation, Family",Join Alice as she chases the White Rabbit into a topsy-turvy world.,2019-11-12,79,min,79,null,1951
s47,Movie,Santa Buddies: The Legend of Santa Paws,Robert Vince,"Christopher Lloyd, Danny Woodburn, Craig Anton, Michael Teigen, Ben Giroux, Paul Rae","United States, Canada","November 5, 2021",2009,G,93 min,"Comedy, Fantasy, Kids","The Buddies remind Santa Paws' feisty son, Puppy Paws, about the true meaning of Christmas.",2021-11-05,93,min,93,null,2009
s266,TV Show,Star Wars Vintage: Clone Wars 2D Micro-Series,null,"Mat Lucas, James Arnold Taylor, Tom Kane, Grey DeLisle, Corey Burton, André Sogliuzzo",null,"April 2, 2021",2021,TV-Y7-FV,1 Season,Series,"Enjoy this thrilling, Emmy® Award-winning animated series that continues the Star Wars story.",2021-04-02,1,season,null,1,2021
s715,Movie,A Goofy Movie,Kevin Lima,"Bill Farmer, Jason Marsden, Jim Cummings, Kellie Martin, Rob Paulsen, Wallace Shawn","United States, Australia, France, Canada","November 12, 2019",1995,G,81 min,"Action-Adventure, Animation, Comedy",Goofy and Max hit the road and get up to their floppy ears in misadventure.,2019-11-12,81,min,81,null,1995


In [0]:
from pyspark.sql import DataFrame

print("5. Normalizando campos multivalorados")

def normalize_multivalue(df_base: DataFrame, id_col: str, src_col: str, out_col: str, out_table: str):
    """
    Cria uma tabela normalizada (N:N) a partir de uma coluna multivalorada separada por vírgula.
    - Remove espaços extras
    - Remove valores vazios
    - Remove duplicidades show_id + valor
    """
    normalized = (
        df_base
        .select(F.col(id_col).alias(id_col), F.col(src_col).alias(src_col))
        .filter(F.col(src_col).isNotNull())
        .withColumn(out_col, F.explode(F.split(F.col(src_col), r",\s*")))
        .withColumn(out_col, F.trim(F.col(out_col)))
        .filter(F.col(out_col).isNotNull() & (F.col(out_col) != ""))
        .select(id_col, out_col)
        .dropDuplicates([id_col, out_col])
    )
    
    (normalized
     .write
     .format("delta")
     .mode("overwrite")
     .saveAsTable(out_table))
    
    return normalized


5. Normalizando campos multivalorados


In [0]:
base = spark.table("silver.disney_titles_clean")

print("- Normalização do campo 'listed_in' (gêneros)")
# Gêneros: listed_in
normalize_multivalue(
    df_base=base,
    id_col="show_id",
    src_col="listed_in",
    out_col="genre",
    out_table="silver.title_genres"
)

print("- Normalização do campo 'country'")
# Países: country
normalize_multivalue(
    df_base=base,
    id_col="show_id",
    src_col="country",
    out_col="country_name",
    out_table="silver.title_countries"
)

print("- Normalização do campo 'director'")
# Diretores: director
normalize_multivalue(
    df_base=base,
    id_col="show_id",
    src_col="director",
    out_col="director_name",
    out_table="silver.title_directors"
)

print("- Normalização do campo 'cast'")
# Elenco: cast
normalize_multivalue(
    df_base=base,
    id_col="show_id",
    src_col="cast",
    out_col="cast_name",
    out_table="silver.title_cast"
)
print("Normalização concluída.")

- Normalização do campo 'listed_in' (gêneros)
- Normalização do campo 'country'
- Normalização do campo 'director'
- Normalização do campo 'cast'
Normalização concluída.


### Validação da quantidade de linhas da camada Silver persistida

Demonstrando que a mesma quantidade de linhas foi mantida

In [0]:
%sql
SELECT COUNT (*) FROM silver.disney_titles_clean

COUNT(*)
1450


### Normalização de Campos Multivalorados (N:N)

O dataset contém campos com múltiplos valores em uma única coluna (separados por vírgula),
o que prejudica análises e modelagem estrela.

Nesta etapa, normalizamos para tabelas auxiliares do tipo:
- show_id + valor_normalizado

Campos normalizados:
- `listed_in` → gêneros (title_genres)
- `country` → países (title_countries)
- `director` → diretores (title_directors)
- `cast` → elenco (title_cast)

In [0]:
from pyspark.sql import DataFrame

print("5. Normalizando campos multivalorados")

def normalize_multivalue(df_base: DataFrame, id_col: str, src_col: str, out_col: str, out_table: str):
    """
    Cria uma tabela normalizada (N:N) a partir de uma coluna multivalorada separada por vírgula.
    - Remove espaços extras
    - Remove valores vazios
    - Remove duplicidades show_id + valor
    """
    normalized = (
        df_base
        .select(F.col(id_col).alias(id_col), F.col(src_col).alias(src_col))
        .filter(F.col(src_col).isNotNull())
        .withColumn(out_col, F.explode(F.split(F.col(src_col), r",\s*")))
        .withColumn(out_col, F.trim(F.col(out_col)))
        .filter(F.col(out_col).isNotNull() & (F.col(out_col) != ""))
        .select(id_col, out_col)
        .dropDuplicates([id_col, out_col])
    )
    
    (normalized
     .write
     .format("delta")
     .mode("overwrite")
     .saveAsTable(out_table))
    
    return normalized



5. Normalizando campos multivalorados


In [0]:
base = spark.table("silver.disney_titles_clean")

print("- Normalização do campo 'listed_in' (gêneros)")
# Gêneros: listed_in
normalize_multivalue(
    df_base=base,
    id_col="show_id",
    src_col="listed_in",
    out_col="genre",
    out_table="silver.title_genres"
)

print("- Normalização do campo 'country'")
# Países: country
normalize_multivalue(
    df_base=base,
    id_col="show_id",
    src_col="country",
    out_col="country_name",
    out_table="silver.title_countries"
)

print("- Normalização do campo 'director'")
# Diretores: director
normalize_multivalue(
    df_base=base,
    id_col="show_id",
    src_col="director",
    out_col="director_name",
    out_table="silver.title_directors"
)

print("- Normalização do campo 'cast'")
# Elenco: cast
normalize_multivalue(
    df_base=base,
    id_col="show_id",
    src_col="cast",
    out_col="cast_name",
    out_table="silver.title_cast"
)
print("Normalização concluída.")

- Normalização do campo 'listed_in' (gêneros)
- Normalização do campo 'country'
- Normalização do campo 'director'
- Normalização do campo 'cast'
Normalização concluída.


### Validações de Qualidade (Silver)

Objetivos:
- confirmar que a Silver canônica foi criada
- medir completude (nulos) por atributo relevante
- confirmar volume das tabelas normalizadas
- identificar possíveis inconsistências em domínios (ex.: type, rating)


In [0]:
%sql
SELECT
  COUNT(*) AS total,
  SUM(CASE WHEN show_id IS NULL THEN 1 ELSE 0 END) AS null_show_id,
  SUM(CASE WHEN type IS NULL THEN 1 ELSE 0 END) AS null_type,
  SUM(CASE WHEN title IS NULL THEN 1 ELSE 0 END) AS null_title,
  SUM(CASE WHEN listed_in IS NULL THEN 1 ELSE 0 END) AS null_listed_in,
  SUM(CASE WHEN release_year_int IS NULL THEN 1 ELSE 0 END) AS null_release_year,
  SUM(CASE WHEN date_added_dt IS NULL THEN 1 ELSE 0 END) AS null_date_added
FROM silver.disney_titles_clean;


total,null_show_id,null_type,null_title,null_listed_in,null_release_year,null_date_added
1450,0,0,0,0,0,3


In [0]:
%sql
SELECT type, COUNT(*) AS qtd
FROM silver.disney_titles_clean
GROUP BY type
ORDER BY qtd DESC;


type,qtd
Movie,1052
TV Show,398


In [0]:
%sql
SELECT 'title_genres' AS tabela, COUNT(*) AS linhas FROM silver.title_genres
UNION ALL
SELECT 'title_countries', COUNT(*) FROM silver.title_countries
UNION ALL
SELECT 'title_directors', COUNT(*) FROM silver.title_directors
UNION ALL
SELECT 'title_cast', COUNT(*) FROM silver.title_cast;


tabela,linhas
title_genres,3910
title_countries,1528
title_directors,1121
title_cast,5899


In [0]:
%sql
SELECT * FROM silver.title_genres

show_id,genre
s678,Action-Adventure
s426,Fantasy
s408,Docuseries
s924,Survival
s399,Family
s233,Science Fiction
s1012,Action-Adventure
s733,Action-Adventure
s931,Kids
s557,Biographical


In [0]:
%sql
SELECT * FROM silver.title_countries

show_id,country_name
s744,United States
s1323,United Kingdom
s1348,United States
s226,United Kingdom
s1117,United States
s261,United States
s1076,United States
s1441,Canada
s1338,United States
s1192,Canada


In [0]:
%sql
SELECT * FROM silver.title_directors

show_id,director_name
s36,Roger Allers
s539,David Ellis
s1276,Art Stevens
s1350,Jun Falkenstein
s49,Robert Vince
s645,Guy Ritchie
s893,Bobby Rubio
s605,Jennifer Lee
s655,John Lasseter
s980,Bruce Hendricks


In [0]:
%sql
SELECT * FROM silver.title_cast

show_id,cast_name
s57,Albert Hall
s268,Cree Summer Francks
s1411,Ken Sansom
s1344,Cole Sprouse
s713,Paul Lukas
s35,Billy Connolly
s33,Avantika
s1181,Kashfia Rahman
s1064,Brian Drummond
s830,Chelsea Staub


### Evidência e Rastreabilidade
As transformações desta camada (tipagem, normalização de duração e normalização relacional) são reproduzíveis a partir deste notebook versionado no GitHub integrado ao Databricks.

As tabelas geradas na Silver serão usadas na Gold para:

- modelo estrela (dimensões + fato)
- análises de diversidade e perfis
- agregações e métricas de negócio

**Observações importantes:**

- Não normalizei `description` porque não é multivalorado e normalmente é usado em NLP/EDA (Gold/ML).
- Não normalizei `rating` porque é categórico simples e já está canônico na Silver.
- Mantive as tabelas normalizadas na camada Silver porque facilitam criar dimensões e pontes na Gold e evitam repetição de parsing/explode em toda query analítica

## 5.4 Camada Gold - Modelagem Analítica e Respostas de Negócio
Construção de tabelas analíticas e viabilizar as respostas às perguntas de negócio definidas no escopo do MVP:

1) Tempo médio entre lançamento e entrada no catálogo (filmes)
2) Evolução da diversidade do catálogo ao longo do tempo (gêneros únicos por ano de adição)
3) Quais gêneros são mais comuns na Disney+



%md
### Pergunta 1
Qual é o tempo médio entre o lançamento e a entrada no catálogo Disney+ para filmes?
